
# GEOS-LDAS Land Variables: Annual Means & Monthly Climatologies

This notebook reads a collection of **GEOS-LDAS land (`lnd`) NetCDF** files, extracts selected variables, computes **annual means** and **monthly climatologies**, and writes them back out to NetCDF files.

**Notes**

- The variables are treated as **means over time** (not totals). For fluxes (e.g., `kg m-2 s-1`, `W m-2`), an *annual mean* is computed; if you need *annual totals*, integrate by time instead of a plain mean.
- The notebook masks large fill values (e.g., `>= 1e14`) as missing before computing statistics.
- Outputs are written as:
  - `annual_means.nc`: one file with a `year` dimension
  - `monthly_climatology.nc`: one file with a `month` dimension


In [2]:
# === Parameters (edit these) ===

from pathlib import Path
import pandas as pd

# Root of your Y####/M## directories:
BASE = Path("/discover/nobackup/projects/land_da/CYGNSS_Experiments/DAv8_M36_cd/DAv8_M36_cd/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg")

# Output folder (created if it doesn't exist)
OUT_DIR = "/discover/nobackup/projects/land_da/CYGNSS_Experiments/DAv8_M36_cd/DAv8_M36_cd/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg"

# Date range
START = pd.Timestamp("2018-08-01")
END   = pd.Timestamp("2024-06-30")

# Variables to extract and process
VARS = [
    "GRN", "LAI",
    "GWETPROF", "GWETROOT", "GWETTOP",
    "PRMC", "RZMC", "SFMC",
    "PRECTOTCORRLAND", "QINFILLAND",
    "SHLAND", "LHLAND", "EVLAND",
]

LND_FILE_GLOB = "DAv8_M36_cd.tavg24_1d_lnd*.*" # broad match to include .nc/.nc4 variants

# Chunking (adjust for your machine/memory)
CHUNKS = {"time": 64, "tile": 10000}

# Compression settings for NetCDF (set zlib=True for smaller files)
ENCODING_COMP = {"zlib": True, "complevel": 3, "shuffle": True}


In [3]:
# === Imports & helpers ===
import re
import os
import numpy as np
import xarray as xr
import pandas as pd
import dask

DATE_RE = re.compile(r"\.(\d{8})_")   # captures YYYYMMDD in filename pieces like .20221222_

FV = 1e15
def clean(ds, name):
    """Mask out large fill values common in GEOS-LDAS outputs."""
    v = ds[name]
    return v.where(v < FV)

def collect_files(base: Path, start: pd.Timestamp, end: pd.Timestamp, file_glob: str):
    files = []
    for ydir in sorted(base.glob("Y*/")):
        for mdir in sorted(ydir.glob("M*/")):
            for f in mdir.glob(file_glob):
                m = DATE_RE.search(f.name)
                if not m:
                    continue
                dt = pd.to_datetime(m.group(1), format="%Y%m%d")
                if start <= dt <= end:
                    files.append(str(f))
    files.sort()
    return files


In [4]:
lnd_files = collect_files(BASE, START, END, LND_FILE_GLOB)
print(f"Found {len(lnd_files)} daily lnd files between {START.date()} and {END.date()}.")
if len(lnd_files) == 0:
    raise SystemExit("No files found. Check BASE/START/END/FILE_GLOB.")
lnd_files[:5]

Found 2161 daily lnd files between 2018-08-01 and 2024-06-30.


['/discover/nobackup/projects/land_da/CYGNSS_Experiments/DAv8_M36_cd/DAv8_M36_cd/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg/Y2018/M08/DAv8_M36_cd.tavg24_1d_lnd_Nt.20180801_1200z.nc4',
 '/discover/nobackup/projects/land_da/CYGNSS_Experiments/DAv8_M36_cd/DAv8_M36_cd/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg/Y2018/M08/DAv8_M36_cd.tavg24_1d_lnd_Nt.20180802_1200z.nc4',
 '/discover/nobackup/projects/land_da/CYGNSS_Experiments/DAv8_M36_cd/DAv8_M36_cd/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg/Y2018/M08/DAv8_M36_cd.tavg24_1d_lnd_Nt.20180803_1200z.nc4',
 '/discover/nobackup/projects/land_da/CYGNSS_Experiments/DAv8_M36_cd/DAv8_M36_cd/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg/Y2018/M08/DAv8_M36_cd.tavg24_1d_lnd_Nt.20180804_1200z.nc4',
 '/discover/nobackup/projects/land_da/CYGNSS_Experiments/DAv8_M36_cd/DAv8_M36_cd/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg/Y2018/M08/DAv8_M36_cd.tavg24_1d_lnd_Nt.20180805_1200z.nc4']

In [5]:
ds = xr.open_mfdataset(
    lnd_files, combine="nested", parallel=True,concat_dim="time",
    decode_times=True, chunks=CHUNKS, engine="netcdf4"
)

# Quick peek
ds

<xarray.Dataset>
Dimensions:              (time: 2161, tile: 70773)
Dimensions without coordinates: time, tile
Data variables: (12/57)
    lon                  (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    lat                  (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    IG                   (time, tile) int32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    JG                   (time, tile) int32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GRN                  (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    LAI                  (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    ...                   ...
    ECHANGELAND          (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    SPSHLAND             (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    SPEVLAND             (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    SPSNLAND             (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    PEATCLSM_WATERLEVEL  (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    PEATCLSM_FSWCHANGE   (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
Attributes:
    CreatedBy:  
    Date:       2025-07-14 at 17:46:19

In [6]:
# Grab the first filename
fname0 = os.path.basename(lnd_files[0])

# Regex to capture the 8-digit date before the "_1200z"
m = re.search(r"\.(\d{8})_1200z", fname0)
if not m:
    raise ValueError(f"No YYYYMMDD date found in {fname0}")
start_str = m.group(1)

# Convert to pandas Timestamp / numpy datetime64
start_date = pd.to_datetime(start_str, format="%Y%m%d")

# Build daily sequence, length = number of files
all_dates = start_date + pd.to_timedelta(np.arange(len(lnd_files)), unit="D")

# Assign to dataset (as datetime64[ns])
ds = ds.assign_coords(time=("time", all_dates.values)).sortby("time")

In [7]:
# Keep only the variables of interest that are actually present
vars_present = [v for v in VARS if v in ds.data_vars]
if not vars_present:
    raise ValueError("None of the requested variables are present in the input files.")

ds = ds[vars_present]

# Mask ridiculous large fill values manually (in case decode didn't mask all)
for v in vars_present:
    da = ds[v]
    ds[v] = xr.where(np.isfinite(da) & (np.abs(da) < 1e14), da, np.nan)

ds

<xarray.Dataset>
Dimensions:          (time: 2161, tile: 70773)
Coordinates:
  * time             (time) datetime64[ns] 2018-08-01 2018-08-02 ... 2024-06-30
Dimensions without coordinates: tile
Data variables: (12/13)
    GRN              (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    LAI              (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETPROF         (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETROOT         (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETTOP          (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    PRMC             (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    ...               ...
    SFMC             (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    PRECTOTCORRLAND  (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    QINFILLAND       (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    SHLAND           (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    LHLAND           (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    EVLAND           (time, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
Attributes:
    CreatedBy:  
    Date:       2025-07-14 at 17:46:19

In [8]:

# Annual means (calendar-year means)
annual_means = ds.groupby("time.year").mean("time", skipna=True, keep_attrs=True)
annual_means = annual_means.rename({"year": "year"})
annual_means.attrs.update(ds.attrs)
annual_means


/gpfsm/dnb34/tdirs/batch/slurm.50249821.amfox/ipykernel_8597/231353254.py:3: UserWarning: rename 'year' to 'year' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  annual_means = annual_means.rename({"year": "year"})


<xarray.Dataset>
Dimensions:          (year: 7, tile: 70773)
Coordinates:
  * year             (year) int64 2018 2019 2020 2021 2022 2023 2024
Dimensions without coordinates: tile
Data variables: (12/13)
    GRN              (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    LAI              (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETPROF         (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETROOT         (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETTOP          (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    PRMC             (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    ...               ...
    SFMC             (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    PRECTOTCORRLAND  (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    QINFILLAND       (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    SHLAND           (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    LHLAND           (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    EVLAND           (year, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
Attributes:
    CreatedBy:  
    Date:       2025-07-14 at 17:46:19

In [9]:

# Monthly climatology across all available years
monthly_climo = ds.groupby("time.month").mean("time", skipna=True, keep_attrs=True)
monthly_climo = monthly_climo.rename({"month": "month"})
monthly_climo.attrs.update(ds.attrs)
monthly_climo


/gpfsm/dnb34/tdirs/batch/slurm.50249821.amfox/ipykernel_8597/2573382421.py:3: UserWarning: rename 'month' to 'month' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  monthly_climo = monthly_climo.rename({"month": "month"})


<xarray.Dataset>
Dimensions:          (month: 12, tile: 70773)
Coordinates:
  * month            (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: tile
Data variables: (12/13)
    GRN              (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    LAI              (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETPROF         (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETROOT         (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    GWETTOP          (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    PRMC             (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    ...               ...
    SFMC             (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    PRECTOTCORRLAND  (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    QINFILLAND       (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    SHLAND           (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    LHLAND           (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
    EVLAND           (month, tile) float32 dask.array<chunksize=(1, 10000), meta=np.ndarray>
Attributes:
    CreatedBy:  
    Date:       2025-07-14 at 17:46:19

In [10]:
# === Grand mean & std across years (from annual_means) ===
grand_mean = annual_means.mean(dim="year", skipna=True, keep_attrs=True)
grand_std  = annual_means.std(dim="year", skipna=True, ddof=1, keep_attrs=True)  # sample std

# Combine into one Dataset with clear suffixes
gm = xr.Dataset()
for v in annual_means.data_vars:
    gm[f"{v}_mean"] = grand_mean[v]
    gm[f"{v}_std"]  = grand_std[v]

# carry over attributes
gm.attrs.update(annual_means.attrs)

gm

<xarray.Dataset>
Dimensions:               (tile: 70773)
Dimensions without coordinates: tile
Data variables: (12/26)
    GRN_mean              (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    GRN_std               (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    LAI_mean              (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    LAI_std               (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    GWETPROF_mean         (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    GWETPROF_std          (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    ...                    ...
    SHLAND_mean           (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    SHLAND_std            (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    LHLAND_mean           (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    LHLAND_std            (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    EVLAND_mean           (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    EVLAND_std            (tile) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    CreatedBy:  
    Date:       2025-07-14 at 17:46:19

In [11]:
def build_encoding(ds_like):
    enc = {}
    for v in ds_like.data_vars:
        enc[v] = dict(ENCODING_COMP)
        enc[v]["dtype"] = "float32"
        enc[v]["_FillValue"] = np.float32(1.0e15)
    return enc

annual_path = os.path.join(OUT_DIR, f"annual_means_{START:%Y%m%d}_{END:%Y%m%d}.nc4")
climo_path  = os.path.join(OUT_DIR, f"monthly_climatology_{START:%Y%m%d}_{END:%Y%m%d}.nc4")
grand_path = os.path.join(OUT_DIR, f"expt_means_{START:%Y%m%d}_{END:%Y%m%d}.nc4")

annual_path

'/discover/nobackup/projects/land_da/CYGNSS_Experiments/DAv8_M36_cd/DAv8_M36_cd/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg/annual_means_20180801_20240630.nc4'

In [12]:
# create delayed writes (do not compute yet)
delayed1 = annual_means.astype("float32").to_netcdf(
    annual_path, format="NETCDF4", encoding=build_encoding(annual_means), compute=False
)
delayed2 = monthly_climo.astype("float32").to_netcdf(
    climo_path, format="NETCDF4", encoding=build_encoding(monthly_climo), compute=False
)
delayed3 = gm.astype("float32").to_netcdf(
    grand_path, format="NETCDF4", encoding=build_encoding(gm), compute=False
)

In [ ]:
# compute all writes in parallel using dask threads (simple, no distributed client)
dask.compute(delayed1, delayed2, delayed3, scheduler="threads")

annual_path, climo_path, grand_path


## (Optional) Annual totals for flux variables

If you need **annual totals** instead of **annual means** for flux variables
(e.g., `kg m-2 s-1` or `W m-2`), integrate over time with actual time-step lengths, e.g.:


In [ ]:

# Example (commented):
# import numpy as np
# flux_vars = ["PRECTOTCORRLAND", "QINFILLAND", "EVLAND", "SHLAND", "LHLAND"]
# sec_per_timestep = np.diff(ds.time.values).astype("timedelta64[s]").astype(int)
# sec_per_timestep = xr.DataArray(np.append(sec_per_timestep, sec_per_timestep[-1]),
#                                 dims=["time"], coords={"time": ds.time})
# tot = {v: (ds[v]*sec_per_timestep).groupby("time.year").sum("time", skipna=True)
#        for v in flux_vars if v in ds}
# annual_totals = xr.Dataset(tot)
# annual_totals.astype("float32").to_netcdf(os.path.join(OUT_DIR, "annual_totals_fluxes.nc"),
#                                           format="NETCDF4",
#                                           encoding=build_encoding(annual_totals))
# annual_totals
